## Modules & Helpful Functions

In [ ]:
import warnings; warnings.filterwarnings('ignore')
import numpy as np,pandas as pd,pylab as pl
import h5py,tensorflow_hub as th
import tensorflow as tf
import skimage.transform as st
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Dropout
from keras.layers import Flatten,Input,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D,GlobalMaxPooling2D
from keras.models import Model
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from keras.callbacks import EarlyStopping

In [ ]:
fpath2='../input/classification-of-handwritten-letters/'
fw='weights.syn.hdf5'
def img_rotate(image,size):
        angle=np.random.randint(-15,15)
        img=st.rotate(image,angle,resize=True)
        img=st.resize(img,(size,size))
        return np.array(img*255,dtype=np.uint8)
def label5symbols(label,symbol):
    length=len(label)
    if length>=5:
        return label
    else:
        no_labels=np.full((5-length),symbol)
        res=np.concatenate((no_labels,label),axis=0)
        return np.array(res,dtype=np.uint8)
def concat5symbols(i,rx,y,size,symbol):
    k=np.random.randint(1,6)
    img=rx[i]; label=y[i]
    for j in range(1,k):
        img=np.concatenate((img,rx[i+j]),axis=1)
        label=np.concatenate((label,y[i+j]),axis=0)
    if k!=5:
        w1=np.random.randint(0,size*5-img.shape[1])
        w2=size*5-img.shape[1]-w1        
        img=np.concatenate((np.zeros((size,w1)),img,
                            np.zeros((size,w2))),axis=1)
    img=np.array(img,dtype=np.uint8)
    label=label5symbols(label,symbol)
    return [img,label,k]

## Synthetic Data #1

In [ ]:
(x_train1,y_train1),(x_test1,y_test1)=\
tf.keras.datasets.mnist.load_data()
y_train1=y_train1.reshape(-1,1)
y_test1=y_test1.reshape(-1,1)
pd.DataFrame([[x_train1.shape,x_test1.shape,x_train1.dtype],
              [y_train1.shape,y_test1.shape,y_train1.dtype]],
             columns=['train','test','dtype'],
             index=['images','labels'])

In [ ]:
rx_train1=np.array([img_rotate(x_train1[i],28)
                    for i in range(x_train1.shape[0])])
rx_test1=np.array([img_rotate(x_test1[i],28)
                   for i in range(x_test1.shape[0])])

In [ ]:
pl.figure(figsize=(3,3))
pl.imshow(rx_train1[100],cmap=pl.cm.summer)
pl.title('example of rotated images \n %s'%y_train1[100]);

In [ ]:
sx_train1=np.empty([rx_train1.shape[0],28,140])
sy_train1=np.empty([y_train1.shape[0],5])
i=0; c=0
while i<rx_train1.shape[0]-5:
    [image,label,k]=\
    concat5symbols(i,rx_train1,y_train1,28,10)
    sx_train1[c]=image; sy_train1[c]=label
    c+=1; i+=k
sx_train1=np.array(sx_train1[:c],dtype=np.uint8)
sy_train1=np.array(sy_train1[:c],dtype=np.uint8)
[sx_train1.shape,sy_train1.shape]

In [ ]:
pl.imshow(sx_train1[100],cmap=pl.cm.summer);
pl.title('example of concatenated images \n %s'%\
         sy_train1[100]);

In [ ]:
sx_test1=np.empty([rx_test1.shape[0],28,140])
sy_test1=np.empty([y_test1.shape[0],5])
i=0; c=0
while i<rx_test1.shape[0]-5:
    [image,label,k]=\
    concat5symbols(i,rx_test1,y_test1,28,10)
    sx_test1[c]=image; sy_test1[c]=label
    c+=1; i+=k
sx_test1=np.array(sx_test1[:c],dtype=np.uint8)
sy_test1=np.array(sy_test1[:c],dtype=np.uint8)
[sx_test1.shape,sy_test1.shape]

In [ ]:
sx_train1=sx_train1.reshape(-1,28,28*5,1)
sx_test1=sx_test1.reshape(-1,28,28*5,1)
n=int(len(sx_test1)/2)
sx_valid1,sy_valid1=sx_test1[:n],sy_test1[:n]
sx_test1,sy_test1=sx_test1[n:],sy_test1[n:]
sy_train1_list=[sy_train1[:,i] for i in range(5)]
sy_valid1_list=[sy_valid1[:,i] for i in range(5)]
sy_test1_list=[sy_test1[:,i] for i in range(5)]
print('Reshape the train set for models')
print(sx_train1.shape,sy_train1_list[0].shape)

## Synthetic Data #2

In [ ]:
f=h5py.File(fpath2+'LetterColorImages_123.h5','r') 
keys=list(f.keys())
letters=u'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
images=np.array(f[keys[1]])/255
images=.7-np.dot(images[...,:3],[.299,.587,.114])**2
images=np.array(images*255,dtype=np.uint8)
labels=np.array(f[keys[2]],dtype=np.uint8).reshape(-1,1)-1
x_train2,x_test2,y_train2,y_test2=\
train_test_split(images,labels,test_size=.2,random_state=1)
pd.DataFrame([[x_train2.shape,x_test2.shape,x_train2.dtype],
              [y_train2.shape,y_test2.shape,y_train2.dtype]],
             columns=['train','test','dtype'],
             index=['images','labels'])

In [ ]:
pl.figure(figsize=(3,3))
pl.imshow(x_train2[100],cmap=pl.cm.summer)
pl.title('example of images \n %s'%\
         letters[y_train2[100][0]]);

In [ ]:
sx_train2=np.empty([x_train2.shape[0],32,160])
sy_train2=np.empty([y_train2.shape[0],5])
i=0; c=0
while i<x_train2.shape[0]-5:
    [image,label,k]=\
    concat5symbols(i,x_train2,y_train2,32,33)
    sx_train2[c]=image; sy_train2[c]=label
    c+=1; i+=k
sx_train2=np.array(sx_train2[:c],dtype=np.uint8)
sy_train2=np.array(sy_train2[:c],dtype=np.uint8)
[sx_train2.shape,sy_train2.shape]

In [ ]:
pl.imshow(sx_train2[100],cmap=pl.cm.summer);
pl.title('example of concatenated images \n %s'%\
         sy_train2[100]);

In [ ]:
sx_test2=np.empty([x_test2.shape[0],32,160])
sy_test2=np.empty([y_test2.shape[0],5])
i=0; c=0
while i<x_test2.shape[0]-5:
    [image,label,k]=\
    concat5symbols(i,x_test2,y_test2,32,33)
    sx_test2[c]=image; sy_test2[c]=label
    c+=1; i+=k
sx_test2=np.array(sx_test2[:c],dtype=np.uint8)
sy_test2=np.array(sy_test2[:c],dtype=np.uint8)
[sx_test2.shape,sy_test2.shape]

In [ ]:
sx_train2=sx_train2.reshape(-1,32,32*5,1)
sx_test2=sx_test2.reshape(-1,32,32*5,1)
n=int(len(sx_test2)/2)
sx_valid2,sy_valid2=sx_test2[:n],sy_test2[:n]
sx_test2,sy_test2=sx_test2[n:],sy_test2[n:]
sy_train2_list=[sy_train2[:,i] for i in range(5)]
sy_valid2_list=[sy_valid2[:,i] for i in range(5)]
sy_test2_list=[sy_test2[:,i] for i in range(5)]
print('Reshape the train set for models')
print(sx_train2.shape,sy_train2_list[0].shape)

## CNN
### Synthetic Data #1

In [ ]:
def syn_model1(size,n):    
    model_input=Input(shape=(size,size*5,1))
    x=BatchNormalization()(model_input)        
    x=Conv2D(32,(5,5),activation='relu',
             padding='same')(model_input)
    x=MaxPooling2D(pool_size=(2,2))(x)     
    x=Conv2D(32,(5,5),activation='relu',
             padding='same')(x)
    x=MaxPooling2D(pool_size=(2,2))(x)    
    x=Dropout(.25)(x)
    x=BatchNormalization()(x)
    x=Conv2D(196,(5,5),activation='relu',
             padding='same')(x) 
    x=MaxPooling2D(pool_size=(2,2))(x) 
    x=Dropout(.25)(x)    
    x=Conv2D(196,(5,5),activation='relu',
             padding='same')(x)
    x=MaxPooling2D(pool_size=(2,2))(x) 
    x=Dropout(.25)(x)              
    x=Flatten()(x)    
    x=Dense(512,activation='relu')(x)    
    x=Dropout(.25)(x)    
    y=[Dense(n,activation='softmax')(x)
       for i in range(5)]
    model=Model(input=model_input,output=y)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',metrics=['accuracy'])
    return model

In [ ]:
syn_model1=syn_model1(28,11)
checkpointer=ModelCheckpoint(filepath=fw,verbose=2,
                             save_best_only=True)
lr_reduction=ReduceLROnPlateau(monitor='val_loss',patience=5,
                               verbose=2,factor=.5)
syn_model1.fit(sx_train1,sy_train1_list, 
               validation_data=(sx_valid1,sy_valid1_list), 
               epochs=30,batch_size=128,verbose=2,
               callbacks=[checkpointer,lr_reduction])

In [ ]:
syn_model1.load_weights(fw)
syn_scores1=syn_model1.evaluate(sx_test1,sy_test1_list,verbose=0)
print("Scores: \n" ,(syn_scores1))
print("First digit. Accuracy: %.2f%%"%(syn_scores1[6]*100))
print("Second digit. Accuracy: %.2f%%"%(syn_scores1[7]*100))
print("Third digit. Accuracy: %.2f%%"%(syn_scores1[8]*100))
print("Fourth digit. Accuracy: %.2f%%"%(syn_scores1[9]*100))
print("Fifth digit. Accuracy: %.2f%%"%(syn_scores1[10]*100))
avg_accuracy1=sum([syn_scores1[i] for i in range(6,11)])/5
print("Synthetic MNIST. Average Accuracy: %.2f%%"%(avg_accuracy1*100))

### Synthetic Data #2

In [ ]:
def syn_model2(size,n):    
    model_input=Input(shape=(size,size*5,1))
    x=BatchNormalization()(model_input)        
    x=Conv2D(32,(5,5),activation='relu',
             padding='same')(model_input)
    x=MaxPooling2D(pool_size=(2,2))(x)     
    x=Conv2D(32,(5,5),activation='relu',
             padding='same')(x)
    x=MaxPooling2D(pool_size=(2,2))(x)    
    x=Dropout(.25)(x)
    x=BatchNormalization()(x)
    x=Conv2D(256,(5,5),activation='relu',
             padding='same')(x) 
    x=MaxPooling2D(pool_size=(2,2))(x) 
    x=Dropout(.25)(x)    
    x=Conv2D(256,(5,5),activation='relu',
             padding='same')(x)
    x=MaxPooling2D(pool_size=(2,2))(x) 
    x=Dropout(.25)(x)              
    x=Flatten()(x)    
    x=Dense(1024,activation='relu')(x)    
    x=Dropout(.25)(x)    
    y=[Dense(n,activation='softmax')(x)
       for i in range(5)]
    model=Model(input=model_input,output=y)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',metrics=['accuracy'])
    return model

In [ ]:
syn_model2=syn_model2(32,34)
checkpointer=ModelCheckpoint(filepath=fw,verbose=2,
                             save_best_only=True)
lr_reduction=ReduceLROnPlateau(monitor='val_loss',patience=5,
                               verbose=2,factor=.5)
estopping=EarlyStopping(monitor='val_loss',patience=20,verbose=2)
syn_model2.fit(sx_train2,sy_train2_list, 
               validation_data=(sx_valid2,sy_valid2_list), 
               epochs=100,batch_size=64,verbose=2,
               callbacks=[checkpointer,lr_reduction,estopping])

In [ ]:
syn_model2.load_weights(fw)
syn_scores2=syn_model2.evaluate(sx_test2,sy_test2_list,verbose=0)
print("Scores: \n" ,(syn_scores2))
print("First letter. Accuracy: %.2f%%"%(syn_scores2[6]*100))
print("Second letter. Accuracy: %.2f%%"%(syn_scores2[7]*100))
print("Third letter. Accuracy: %.2f%%"%(syn_scores2[8]*100))
print("Fourth letter. Accuracy: %.2f%%"%(syn_scores2[9]*100))
print("Fifth letter. Accuracy: %.2f%%"%(syn_scores2[10]*100))
avg_accuracy2=sum([syn_scores2[i] for i in range(6,11)])/5
print("Synthetic Letters. Average Accuracy: %.2f%%"%(avg_accuracy2*100))